<a href="https://colab.research.google.com/github/Levy95/Progetto-Corso-Reti-Neurali/blob/main/Progetto_Corso_Reti_Neurali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Device: serve per ottimizzare la computazione di calcoli sulle matrici
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Dataset: MNIST.
train_dataset_mnist = torchvision.datasets.MNIST(root='./data',
                                           train = True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset_mnist = torchvision.datasets.MNIST(root='./data',
                                                train = False,
                                                transform = transforms.ToTensor())

Segue un'immagine estratta dal training-set del datset MNIST.

In [ ]:
image, label = train_dataset_mnist[0]
plt.imshow(image[0], cmap='gray')
print('Label:', label)

In [ ]:
# Dataset: FashionMNIST
train_dataset_fashionmnist = torchvision.datasets.FashionMNIST(root = './data',
                                                               train = True,
                                                               transform = transforms.ToTensor(),
                                                               download = True)
test_dataset_fashionmnist = torchvision.datasets.FashionMNIST(root='./data',
                                                              train = True,
                                                              transform = transforms.ToTensor())                          

Segue un'immagine estratta dal training-set del datset FashionMNIST.

In [ ]:
image, label = train_dataset_fashionmnist[0]
plt.imshow(image[0], cmap='gray')
print('Label:', label)

In [ ]:
# Dataset: CIFAR10
train_dataset_cifar10 = torchvision.datasets.CIFAR10(root = './data',
                                                      train = True,
                                                      transform = transforms.ToTensor(),
                                                      download = True)
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root = './data',
                                                    train = False,
                                                    transform = transforms.ToTensor())                                                                                                   

Segue un'immagine estratta dal training-set del datset CIFAR10.

In [ ]:
image, label = train_dataset_cifar10[0]
plt.imshow(image[0], cmap='brg')
print('Label:', label)

Seguono, nell'ordine, le classi che rappresentano l'implementazione della struttura della rete Feed-Forward con uno e due layer intermedi.

In [6]:
# Rete Neurale Feed-Forward completamente connessa con un solo layer: Shallow Network
class ReteNeurale1(nn.Module):
  def __init__(self,input_size, hidden_size, num_classi, act_func=1):
   super(ReteNeurale1, self).__init__()
   self.input_size = input_size
   self.l1 = nn.Linear(input_size, hidden_size)
   self.l2 = nn.Linear(hidden_size, num_classi)
   if act_func == 1:
    self.actfunc = nn.ReLU()
   elif act_func == 2:
    self.actfunc = nn.LeakyReLU()
   elif act_func == 3:
     self.actfunc = nn.Sigmoid() 
   

  def forward(self, x):
     out = self.l1(x)
     out = self.actfunc(out)
     out = self.l2(out)
     return out

In [7]:
# Rete Neurale Feed-Forward completamente connessa con due layer
class ReteNeurale2(nn.Module):
  def __init__(self, input_size, hidden_size1, hidden_size2, num_classi, act_func=1):
    super(ReteNeurale2, self).__init__()
    self.input_size = input_size
    self.l1 = nn.Linear(input_size, hidden_size1)
    self.l2 = nn.Linear(hidden_size1, hidden_size2)
    self.l3 = nn.Linear(hidden_size2, num_classi)
    if act_func == 1:
      self.actfunc = nn.ReLU()
    elif act_func == 2:
      self.actfunc = nn.LeakyReLU()
    elif act_func == 3:
      self.actfunc = nn.Sigmoid()
  
  def forward(self, x):
    out = self.l1(x)
    out = self.actfunc(out)
    out = self.l2(out)
    out = self.actfunc(out)
    out = self.l3(out)
    return out

In [11]:
# Funzione che implementa il training di una rete neurale che compie classificazione di immagini.

def train_function(rete, load_tr_ds, opt, loss_fun, img_size, num_ep):
  # Inputs:
  # rete: istanza di una classe di rete neurale
  # load_tr_ds: training (loader) set del dataset prescelto
  # opt: algoritmo di learning
  # img_size: dimensione delle immagini
  # num_ep: numero di epoche di training
  # Output: loss
  n_tot_steps = len(load_tr_ds) # step = len(load_tr_ds)/batch_size
  loss_vec = [] # vettore in cui si salvano i valori della loss ad ogni step, serve per il plot
  for epoch in range(num_ep):
    for i, (immagini, etichette) in enumerate(load_tr_ds):
      # si coverte il formato dei dati
      immagini = immagini.reshape(-1, img_size).to(device)
      etichette = etichette.to(device)

      # Forward step: si passano i dati attraverso i layer
      outputs = rete(immagini)
      # Si calcola la loss
      loss = loss_fun(outputs, etichette)
      
      # Training step: si calcola il gradiente della loss e si compie un passo 
      # dell'algoritnmo di training
      opt.zero_grad()
      loss.backward()
      opt.step()
       
      # si stampano i valori della loss 
      if (i+1)%100 == 0:
        print (f'Epoch [{epoch+1}/{num_ep}], Step [{i+1}/{n_tot_steps}], Loss: {loss.item():.4f}') 
        loss_vec.append(loss.item()) # salvataggio dei valori della loss ad ogni step
        #if loss <= 0.04: # early-stopping
         # break
        
  return loss_vec

In [12]:
# Funzione che testa una rete neurale neurale allenata per fare classificazione di immagini

def test_function(rete, load_ts_ds, img_size):
  # Input:
  # rete: istanza di una classe di rete neurale
  # lead_ts_ds: test (loader) set del dataset prescelto
  # img_size: dimensione delle immagini
  with torch.no_grad():
    num_corrette = 0
    num_campioni = 0
    for immagini, etichette in load_ts_ds:
      immagini = immagini.reshape(-1, img_size).to(device)
      etichette = etichette.to(device)
      output = rete(immagini)

      # max ha come output una tupla del tipo (valore, indice)
      _, pred = torch.max(output.data, 1)
      num_campioni += etichette.size(0)
      num_corrette += (pred == etichette).sum().item()

    # accuratezza della rete: percentuali di immagini etichettate correttamente sul campione di dati
    acc = 100 * num_corrette/num_campioni
    print(f'Accuracy della rete su 10000 immagini test: {acc} %')
    return acc

In [19]:
# Iperparametri
input_sizeA = 784 # per immagini 28x28 in scale di grigio
input_sizeB = 3072 # per immagini 32x32 in scale RGB
img_sizeA = 784
img_sizeB = 3072
hidden_size1 = 1000
hidden_size2 = 500
num_classi = 10
learning_rate = 0.0001
batch_size = 100

# Si creano i data-loaders: servono per caricare i dati in batches di dimensioni prestabilite
train_loader_mnist = torch.utils.data.DataLoader(dataset=train_dataset_mnist,
                                           batch_size=batch_size,
                                           shuffle = True)
test_loader_mnist = torch.utils.data.DataLoader(dataset= test_dataset_mnist,
                                          batch_size=batch_size,
                                          shuffle=False)

train_loader_fashionmnist = torch.utils.data.DataLoader(dataset = train_dataset_fashionmnist,
                                                        batch_size = batch_size,
                                                        shuffle=True)
test_loader_fashionmnist = torch.utils.data.DataLoader(dataset = test_dataset_fashionmnist,
                                                       batch_size = batch_size,
                                                       shuffle = False)

train_loader_cifar10 = torch.utils.data.DataLoader(dataset = train_dataset_cifar10,
                                                   batch_size = batch_size,
                                                   shuffle = True)
test_loader_cifar10 = torch.utils.data.DataLoader(dataset = test_dataset_cifar10,
                                                  batch_size = batch_size,
                                                  shuffle = False)

Testiamo la rete neurale con un solo layer intermedio sui dataset MNIST e FashionMNIST.

In [24]:
# Instanziare la rete neurale scegliendo quale funzione di attivazione si vuole usare
rete1 = ReteNeurale1(input_sizeA, hidden_size1, num_classi, act_func=2).to(device)

# Scegliere l'algoritmo di learning e la funzione di output e loss
alg = torch.optim.Adam(rete1.parameters(), lr = learning_rate)
F = nn.CrossEntropyLoss()

In [ ]:
print('Dataset MNIST:\n')
l1 = train_function(rete1, train_loader_mnist, alg, F, img_sizeA, 2)
a1 = test_function(rete1, test_loader_mnist, img_sizeA) 
print('\n')

Esempio di come si sono eseguiti i grafici relativi al variare della loss durante la fase di training.

In [ ]:
x = np.linspace(0, 2, 12)
y = l1
plt.plot(x,y)

In [ ]:
print('Dataset FashionMNIST:\n')
l2 = train_function(rete1, train_loader_fashionmnist, alg, F, img_sizeA, 2)
a2 = test_function(rete1, test_loader_fashionmnist, img_sizeA)
print('\n')

Testiamo la rete neurale con un solo layer sul dataset CIFAR10. E' necessario creare una nuovo oggetto-rete poichè le dimensioni delle immagini di CIFAR10 sono diverse da quelle degli altri due dataset.

In [ ]:
rete1B = ReteNeurale1(input_sizeB, hidden_size1, num_classi, act_func=1).to(device)
alg = torch.optim.Adam(rete1B.parameters(), lr = learning_rate)
print('Dataset CIFAR10:\n')
l3 = train_function(rete1B, train_loader_cifar10, alg, F, img_sizeB, 2)
a3 = test_function(rete1B, test_loader_cifar10, img_sizeB)

Allo stesso modo ora testiamo la rete con due layer

In [ ]:
rete2 = ReteNeurale2(input_sizeA, hidden_size1, hidden_size2, num_classi, act_func=1).to(device)
alg = torch.optim.Adam(rete2.parameters(), lr=learning_rate)

print('Dataset MNIST:\n')
l1b = train_function(rete2, train_loader_mnist, alg, F, img_sizeA, 2)
a1 = test_function(rete2, test_loader_mnist, img_sizeA) 

In [ ]:
x = np.linspace(0, 2, 12)
y = l1b
plt.plot(x,y)

In [ ]:
print('Dataset FashionMNIST:\n')
l2b = train_function(rete2, train_loader_fashionmnist, alg, F, img_sizeA, 2)
a2 = test_function(rete2, test_loader_fashionmnist, img_sizeA)

In [ ]:
rete2B = ReteNeurale2(input_sizeB, hidden_size1, hidden_size2, num_classi, act_func=1).to(device)
alg = torch.optim.Adam(rete2B.parameters(), lr = learning_rate)
print('Dataset CIFAR10:\n')
l3b = train_function(rete2B, train_loader_cifar10, alg, F, img_sizeB, 2)
a3 = test_function(rete2B, test_loader_cifar10, img_sizeB)